## Cat Classifier
Determines whether an image contains a cat<br>
Dataset taken from Andrew Ng's Coursera course<br>
Last update: 12/23/23

In [1]:
import cupy as np # Use Cuda
from h5py import File
from importlib import reload
from PIL import Image
from sandbox import model, layers, activations, costs, utils

In [2]:
# Load data
train_dataset = File('dataset\\train_catvnoncat.h5', 'r')
train_x = np.array(train_dataset['train_set_x'][:]) # Train set features
train_y = np.array(train_dataset['train_set_y'][:]) # Train set labels

test_dataset = File('dataset\\test_catvnoncat.h5', 'r')
test_x = np.array(test_dataset['test_set_x'][:]) # Test set features
test_y = np.array(test_dataset['test_set_y'][:]) # Test set labels

train_y = train_y.reshape((train_y.shape[0], 1))
test_y = test_y.reshape((test_y.shape[0], 1))

# Flatten and normalize
train_x = train_x.reshape(train_x.shape[0], -1) / 255
test_x = test_x.reshape(test_x.shape[0], -1) / 255

In [3]:
reload(model)
np.random.seed(1)

cat = model.Model(cuda=True)
cat.add(layers.Dense(units=20, activation=activations.ReLU()))
cat.add(layers.Dense(units=7, activation=activations.ReLU()))
cat.add(layers.Dense(units=5, activation=activations.ReLU()))
cat.add(layers.Dense(units=1, activation=activations.Sigmoid()))

cat.configure(cost_type=costs.BinaryCrossentropy())
cat.train(train_x, train_y, learning_rate=0.0075, epochs=200, batch_size=32, verbose=True)
cat.save() # Save model parameters

Cost on epoch 0: 0.68793
Cost on epoch 20: 0.49068
Cost on epoch 40: 0.50392
Cost on epoch 60: 0.31861
Cost on epoch 80: 0.45292
Cost on epoch 100: 0.47742
Cost on epoch 120: 0.43655
Cost on epoch 140: 0.19214
Cost on epoch 160: 0.0834
Cost on epoch 180: 0.06103
Cost on epoch 200: 0.01977


In [5]:
# Print model summary
cat.summary()

# Assess model accuracy
pred_train = utils.binary_round(cat.predict(train_x)) # Get model accuracy on training data
print('\nTraining Accuracy: '  + str(np.round(np.sum((pred_train == train_y)/train_x.shape[0]), decimals=5)))
pred_test = utils.binary_round(cat.predict(test_x)) # Get model accuracy on testing data
print('Testing Accuracy: '  + str(np.round(np.sum((pred_test == test_y)/test_x.shape[0]), decimals=5)))

# Use model on custom image
my_image = 'dataset\\cat.jpg' 
num_px = 64

image = np.array(Image.open(my_image).resize((num_px, num_px))) / 255 # Resize and normalize image, cast to NumPy array
image = image.reshape((1, num_px * num_px * 3)) # Flatten image array
my_predicted_image = utils.binary_round(cat.predict(image)) # Predict custom image
print(f"Image guess: {my_predicted_image.item()}")

+------------+------------+
| Layer type | Parameters |
+------------+------------+
|   Dense    |   245780   |
|   Dense    |    147     |
|   Dense    |     40     |
|   Dense    |     6      |
+------------+------------+
Total parameters: 245973

Training Accuracy: 0.98565
Testing Accuracy: 0.72
Image guess: 1
